In [3]:
!pip install fastapi google-genai

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached websockets-14.2-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/94.9 kB ? eta -:--:--
   ------------ --------------------------- 30.7/94.9 kB ? eta -:--:--
   ----------------- ---------------------- 41.0/94.9 kB 991.0 kB/s eta 0:00:01
   ------------------------- -------------- 61.4/94.9 kB 469.7 kB/s eta 0:00:01
   ---------------------------------------- 94.9/94.9 kB 600.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/141.0 kB ? eta -:--:--
   ----------------------- ---------------- 81.9/141.0 kB 4.8 MB/s eta 0:00:01
   ------------------------------- -------- 112.6/141.0 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 141.0/141.0 kB 1.2 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
   ---------------------------------------- 0


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from google import genai
from google.genai import types
import shutil
from dotenv import load_dotenv
import os
from pydantic import BaseModel

In [2]:
load_dotenv()

True

In [3]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [24]:
import subprocess

def ffmpeg_runner(ffmpeg_code: str):
    """
    Runs an FFmpeg command and prints output in real-time.
    """
    try:
        process = subprocess.run(
            ffmpeg_code, 
            shell=True, 
            check=True, 
            stdout=subprocess.PIPE,  # Capture standard output
            stderr=subprocess.STDOUT,  # Capture errors in the same stream
            text=True  # Ensure output is treated as text (not bytes)
        )
        
        print(process.stdout)  # Print FFmpeg output
        return True
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg failed with error:\n{e.output}")
        return False

In [25]:
SYSTEM_INSTRUCTION = """
Write ffmpeg code for the queries given. The files are inside `../files/upload`. Save the file inside `../files/edit` as meow.mp4.

example code: ffmpeg -i input1.mp4 -i input2.mp4 -filter_complex "[0:v:0][0:a:0][1:v:0][1:a:0]concat=n=2:v=1:a=1[outv][outa]" -map "[outv]" -map "[outa]" output.mp4
"""

chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=SYSTEM_INSTRUCTION,
        tools=[ffmpeg_runner],
        
    ),
)

In [62]:
response = chat.send_message("merge overlay.mp4 and the first 2 seconds of output.mp4")

ffmpeg version N-118616-g3e9777dc75-20250304 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (crosstool-NG 1.27.0.18_7458341)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-libxml2 --enable-lzma --enable-fontconfig --enable-libharfbuzz --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --enable-libdvdread --enable-libdvdnav --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --enable-libaribcaption --enable-libass --enable-libbluray --enable-lib

In [35]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='I apologize, but I was unable to merge the videos using the tools available to me.')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=-0.050273236301210195, finish_reason=<FinishReason.STOP: 'STOP'>, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)], create_time=None, response_id=None, model_version='gemini-2.0-flash', prompt_feedback=None, usage_metadata=GenerateContentResponseUsageMetadata(cached_content_token_count=None, candidates_token_count=18, prompt_token_count=599, total_token_count=617), automatic_function_calling_history=[UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None

In [63]:
response.automatic_function_calling_history[-1].parts[0].function_response.response['result']

True

In [8]:
import subprocess

def ffmpeg_runner(ffmpeg_code: str):
    print(ffmpeg_code)
    try:
        process = subprocess.run(
            ffmpeg_code, 
            shell=True, 
            check=True, 
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT, 
            text=True
        )
        print(process.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg failed with error:\n{e.output}")
        return False


In [16]:
path = r"""C:/Users/Robin Roy/Desktop/idkhack/files"""

In [17]:
ffmpeg_code = fr"""ffmpeg -i '{path}/version1.mp4' -i '{path}/version2.mp4' -filter_complex "[0:v:0][0:a:0][1:v:0][1:a:0]concat=n=2:v=1:a=1[outv][outa]" -map "[outv]" -map "[outa]" -c:v libx264 -crf 23 -preset fast -c:a aac -b:a 192k '{path}/version13.mp4'"""

In [18]:
ffmpeg_runner(ffmpeg_code)

ffmpeg -i 'C:/Users/Robin Roy/Desktop/idkhack/files/version1.mp4' -i 'C:/Users/Robin Roy/Desktop/idkhack/files/version2.mp4' -filter_complex "[0:v:0][0:a:0][1:v:0][1:a:0]concat=n=2:v=1:a=1[outv][outa]" -map "[outv]" -map "[outa]" -c:v libx264 -crf 23 -preset fast -c:a aac -b:a 192k 'C:/Users/Robin Roy/Desktop/idkhack/files/version13.mp4'
FFmpeg failed with error:
ffmpeg version N-118616-g3e9777dc75-20250304 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (crosstool-NG 1.27.0.18_7458341)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-libxml2 --enable-lzma --enable-fontconfig --enable-libharfbuzz --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-l

False

In [ ]:
ffmpeg -i '../files/version1.mp4' -i '../files/version2.mp4' -filter_complex "[0:v:0][0:a:0][1:v:0][1:a:0]concat=n=2:v=1:a=1[outv][outa]" -map "[outv]" -map "[outa]" -c:v libx264 -crf 23 -preset fast -c:a aac -b:a 192k '../files/version13.mp4'

In [19]:
path = r"C:/Users/Robin Roy/Desktop/idkhack/files"

ffmpeg_code = fr"""ffmpeg -i "{path}/version1.mp4" -i "{path}/version2.mp4" -filter_complex "[0:v:0][0:a:0][1:v:0][1:a:0]concat=n=2:v=1:a=1[outv][outa]" -map "[outv]" -map "[outa]" -c:v libx264 -crf 23 -preset fast -c:a aac -b:a 192k "{path}/version13.mp4" """

ffmpeg_runner(ffmpeg_code)

ffmpeg -i "C:/Users/Robin Roy/Desktop/idkhack/files/version1.mp4" -i "C:/Users/Robin Roy/Desktop/idkhack/files/version2.mp4" -filter_complex "[0:v:0][0:a:0][1:v:0][1:a:0]concat=n=2:v=1:a=1[outv][outa]" -map "[outv]" -map "[outa]" -c:v libx264 -crf 23 -preset fast -c:a aac -b:a 192k "C:/Users/Robin Roy/Desktop/idkhack/files/version13.mp4" 
ffmpeg version N-118616-g3e9777dc75-20250304 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (crosstool-NG 1.27.0.18_7458341)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-libxml2 --enable-lzma --enable-fontconfig --enable-libharfbuzz --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --en

True

In [ ]:
import os
from google import genai
from google.genai import types
import time


def generate(filename: str):
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    file =  client.files.upload(file=filename)

    while file.state.name == "PROCESSING":
        print('Waiting for video to be processed.')
        time.sleep(10)
        file = client.files.get(name=file.name)

    if file.state.name == "FAILED":
        raise ValueError(file.state.name)
    print(f'Video processing complete: ' + file.uri)

    model = "gemini-2.0-flash"

    chat = client.chats.create(
        model=model,
        config=types.GenerateContentConfig(system_instruction="""Do audio description on the provided video file. Return the output as a proper subtitle srt file within 3 backticks (```).

        Example:

        ```
        1
00:00:01,200 --> 00:00:16,200
The scene is a person looking at the sky

2
00:00:17,200 --> 00:00:25,100
Sky full of stars glowing is shown
        ```                                           
"""),
    )

    response = chat.send_message(message=types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=file.uri,
                    mime_type=file.mime_type,
                ),
                types.Part.from_text(text="Do audio description on this. remember to return with proper timestamps formatted within 3 backticks (```)")
            ],
        ),)
    
    return response

In [40]:
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
response = generate(r"""C:\Users\Robin Roy\Videos\troll\DIY TICTAC Fidget Spinner _ Unlimited Spinning _ How To _.mp4""")

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/9gpds0ngb4rj


In [48]:
print(response.text.split("```")[1])


1
00:00:00,324 --> 00:00:04,514
A person with black hair, wearing glasses, talks to the viewer, with a white brick wall in the background.

2
00:00:04,908 --> 00:00:10,308
A low angle shot of a white church with 3 towers and crosses on top of each.

3
00:00:11,155 --> 00:00:17,345
A green box of Tic Tac candy, a tire with its wheel set, and a nail are on a gray concrete surface.

4
00:00:17,345 --> 00:00:29,715
A hand picks up the green Tic Tac box and pokes 2 holes in the side using the nail.

5
00:00:32,245 --> 00:00:40,145
A close up of a green Tic Tac box with holes poked in both sides is shown.

6
00:00:41,085 --> 00:00:47,175
A hand attaches the wheel set to one side of the Tic Tac box.

7
00:00:47,755 --> 00:00:55,335
A hand grabs 2 coins, and a glue bottle is present.

8
00:00:55,825 --> 00:01:17,755
The hand places drops of glue on the corners of the Tic Tac box, attaching the coins on top of the glue.

9
00:01:17,755 --> 00:01:26,785
The coins have been glued, and the hand i

In [ ]:
GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='```\n1\n00:00:00,324 --> 00:00:04,514\nA person with black hair, wearing glasses, talks to the viewer, with a white brick wall in the background.\n\n2\n00:00:04,908 --> 00:00:10,308\nA low angle shot of a white church with 3 towers and crosses on top of each.\n\n3\n00:00:11,155 --> 00:00:17,345\nA green box of Tic Tac candy, a tire with its wheel set, and a nail are on a gray concrete surface.\n\n4\n00:00:17,345 --> 00:00:29,715\nA hand picks up the green Tic Tac box and pokes 2 holes in the side using the nail.\n\n5\n00:00:32,245 --> 00:00:40,145\nA close up of a green Tic Tac box with holes poked in both sides is shown.\n\n6\n00:00:41,085 --> 00:00:47,175\nA hand attaches the wheel set to one side of the Tic Tac box.\n\n7\n00:00:47,755 --> 00:00:55,335\nA hand grabs 2 coins, and a glue bottle is present.\n\n8\n00:00:55,825 --> 00:01:17,755\nThe hand places drops of glue on the corners of the Tic Tac box, attaching the coins on top of the glue.\n\n9\n00:01:17,755 --> 00:01:26,785\nThe coins have been glued, and the hand is placing the wheel set back into the box.\n\n10\n00:01:27,555 --> 00:01:32,785\nA person spins the fidget spinner, which is green with 2 gray coins and a wheel set in the middle.\n```')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=-0.5703954282014266, finish_reason=<FinishReason.STOP: 'STOP'>, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)], create_time=None, response_id=None, model_version='gemini-2.0-flash', prompt_feedback=None, usage_metadata=GenerateContentResponseUsageMetadata(cached_content_token_count=None, candidates_token_count=506, prompt_token_count=27213, total_token_count=27719), automatic_function_calling_history=[], parsed=None)

In [49]:
!pip install git+https://github.com/suno-ai/bark.git

  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git 'C:\Users\Robin Roy\AppData\Local\Temp\pip-req-build-f6ar43ud'

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Cloning https://github.com/suno-ai/bark.git to c:\users\robin roy\appdata\local\temp\pip-req-build-f6ar43ud
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached encodec-0.1.1-py3-none-any.whl
     ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
     ---------------------------------------- 44.0/44.0 kB 2.1 MB/s eta 0:00:00
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 1.0 MB/s eta 0:00:00
   -------------------------------------